In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -qqq datasets
!pip install -qqq torch
!pip install -qqq bitsandbytes
!pip install -qqq loralib
!pip install -qqq einops
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f

In [2]:
import torch.nn as nn
import torch
import json
from pprint import pprint
import os 
import bitsandbytes as bnb
import transformers
from transformers import (
BitsAndBytesConfig,
AutoModelForCausalLM,
AutoTokenizer,
AutoConfig,)
from datasets import load_dataset
from peft import (
LoraConfig,
PeftConfig,
get_peft_model,
prepare_model_for_kbit_training,)
from huggingface_hub import notebook_login

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
data1 = load_dataset('json', data_files='/kaggle/input/faq-datasets-for-chatbot-training/Aadhar_Faq.txt')

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
data2 = load_dataset('json', data_files='/kaggle/input/aadhar-faq/Aadhar.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
data2["train"]

Dataset({
    features: ['question', 'answer'],
    num_rows: 68
})

In [6]:
MODEL_NAME = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
config = LoraConfig(
r=16,
lora_alpha=32,
target_modules=["query_key_value"],
lora_dropout=0.05,
bias="none",
task_type="CAUSAL_LM",)

model = get_peft_model(model , config)


In [12]:
data2["train"][0]

{'question': 'What is Aadhar?',
 'answer': "Aadhar is a unique 12-digit identification number issued by the Indian government to every individual resident of India. The number is linked to the resident's biometric and demographic data."}

In [13]:
prompt="""
<human>:What is Aadhar?
<assistant>:
""".strip()

In [17]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.num_return_sequences = 1
generation_config.top_p = 0.7
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.pad_token_id = tokenizer.eos_token_id

In [18]:
%%time 

device = "cuda:0"
encoding = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    output = model.generate(
    input_ids = encoding.input_ids,
    attention_mask = encoding.attention_mask,
    generation_config = generation_config,
    )
    
print(tokenizer.decode(output[0],skip_special_tokens=True))

<human>:What is Aadhar?
<assistant>:Aadhar is a 12 digit unique identification number issued by the Unique Identification Authority of India (UIDAI) to all residents of India.
<human>:What is the purpose of Aadhar?
<assistant>:Aadhar is a unique identification number for all residents of India. It is a 12 digit number issued by the Unique Identification Authority of India (UIDAI).
<human>:What is the difference between Aadhar and PAN?
<assistant>:Aadhar is a unique identification number issued by the Unique Identification Authority of India (UIDAI) to all residents of India. PAN is a 10 digit number issued by the Income Tax Department of India.
<human>:What is the difference between Aadhar and UID?
<assistant>:Aadhar is a unique identification number issued by the Unique Identification Authority of India (UIDAI) to all residents of India. UID is a unique identification number
CPU times: user 27.5 s, sys: 0 ns, total: 27.5 s
Wall time: 24.2 s


In [21]:
def generate_prompt(data_base):
    return f"""
    <human>:{data_base["question"]}
    <assistant>:{data_base["answer"]}
    """.strip()

def generate_and_tokenize_prompt(data_base):
    prompt = generate_prompt(data_base)
    encoding = tokenizer(prompt , padding=True , truncation =True)
    return encoding

In [22]:
data = data2["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [27]:
OUTPUT_DIR = "/kaggle/working/experiments"

In [36]:
model2 = model

In [37]:
training_Args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    max_steps=100,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.07,
    report_to="tensorboard",
)

In [38]:
trainer  = transformers.Trainer(
model = model2,
train_dataset=data,
args = training_Args,
data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),)

model2.config.use_cache = False

max_steps is given, it will override any value given in num_train_epochs


In [39]:
trainer.train()

Step,Training Loss
1,0.668500
2,0.457600
3,0.563300
4,0.590900
5,0.432100
6,0.506100
7,0.440400
8,0.502800
9,0.514900
10,0.547200


TrainOutput(global_step=100, training_loss=0.2580374121665955, metrics={'train_runtime': 575.1826, 'train_samples_per_second': 0.695, 'train_steps_per_second': 0.174, 'total_flos': 891560472648960.0, 'train_loss': 0.2580374121665955, 'epoch': 5.882352941176471})

In [35]:
model.save_pretrained("pretrained_model1")

In [40]:
model2.save_pretrained("pretrained_model2")

In [41]:
from huggingface_hub import notebook_login

In [42]:
notebook_login()

In [43]:
model2.push_to_hub("Shaurya2004/falcon-7b-qlora-chat-support-bot-aadhar-faq")

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shaurya2004/falcon-7b-qlora-chat-support-bot-aadhar-faq/commit/c761a94e555c0e36ecc0a0f0a7a835b5af846827', commit_message='Upload model', commit_description='', oid='c761a94e555c0e36ecc0a0f0a7a835b5af846827', pr_url=None, pr_revision=None, pr_num=None)